In [28]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import kurtosis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from hyperopt import fmin, tpe, hp, Trials

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体，或者使用你系统上可用的其他字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

In [42]:
table = pd.read_parquet('ag\AG2406_20240403.parquet')

In [43]:
table

,order_book_id,exchange_time,trading_date,open,last,high,low,prev_settlement,prev_close,Volume,OpenInterest,Turnover,limit_up,limit_down,AskPrice1,AskPrice2,AskPrice3,AskPrice4,AskPrice5,BidPrice1,BidPrice2,BidPrice3,BidPrice4,BidPrice5,AskVolume1,AskVolume2,AskVolume3,AskVolume4,AskVolume5,BidVolume1,BidVolume2,BidVolume3,BidVolume4,BidVolume5,change_rate
0,AG2406,2024-04-02 20:59:00.500,2024-04-03,6590.0,6590.0,6590.0,6590.0,6474.0,6562.0,1392.0,586974.0,1.375992e+08,6927.0,6020.0,6591.0,6592.0,6593.0,6594.0,6595.0,6590.0,6589.0,6588.0,6587.0,6586.0,5.0,9.0,19.0,63.0,255.0,3.0,103.0,170.0,26.0,148.0,0.017918
1,AG2406,2024-04-02 21:00:00.500,2024-04-03,6590.0,6592.0,6595.0,6590.0,6474.0,6562.0,1945.0,587315.0,1.922808e+08,6927.0,6020.0,6595.0,6596.0,6597.0,6598.0,6599.0,6592.0,6591.0,6590.0,6589.0,6588.0,263.0,39.0,66.0,65.0,90.0,5.0,214.0,45.0,26.0,150.0,0.018227
2,AG2406,2024-04-02 21:00:01.000,2024-04-03,6590.0,6593.0,6595.0,6590.0,6474.0,6562.0,2792.0,587143.0,2.760460e+08,6927.0,6020.0,6594.0,6595.0,6596.0,6597.0,6598.0,6593.0,6591.0,6590.0,6589.0,6588.0,39.0,112.0,44.0,82.0,92.0,3.0,90.0,70.0,62.0,150.0,0.018381
3,AG2406,2024-04-02 21:00:01.500,2024-04-03,6590.0,6594.0,6595.0,6590.0,6474.0,6562.0,3140.0,587218.0,3.104635e+08,6927.0,6020.0,6595.0,6596.0,6597.0,6598.0,6599.0,6594.0,6593.0,6592.0,6591.0,6590.0,97.0,47.0,102.0,93.0,141.0,130.0,38.0,28.0,171.0,84.0,0.018536
4,AG2406,2024-04-02 21:00:02.000,2024-04-03,6590.0,6594.0,6596.0,6590.0,6474.0,6562.0,3533.0,587227.0,3.493367e+08,6927.0,6020.0,6595.0,6596.0,6597.0,6598.0,6599.0,6594.0,6593.0,6592.0,6591.0,6590.0,1.0,35.0,103.0,105.0,155.0,57.0,5.0,18.0,197.0,67.0,0.018536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65774,AG2406,2024-04-03 14:59:58.000,2024-04-03,6590.0,6791.0,6798.0,6541.0,6474.0,6562.0,1357977.0,602704.0,1.357056e+11,6927.0,6020.0,6792.0,6793.0,6794.0,6795.0,6796.0,6791.0,6790.0,6789.0,6788.0,6787.0,15.0,24.0,8.0,45.0,181.0,1.0,47.0,31.0,41.0,15.0,0.048965
65775,AG2406,2024-04-03 14:59:58.500,2024-04-03,6590.0,6791.0,6798.0,6541.0,6474.0,6562.0,1358033.0,602687.0,1.357113e+11,6927.0,6020.0,6791.0,6793.0,6794.0,6795.0,6796.0,6790.0,6789.0,6788.0,6787.0,6786.0,81.0,35.0,8.0,45.0,167.0,44.0,30.0,41.0,15.0,76.0,0.048965
65776,AG2406,2024-04-03 14:59:59.000,2024-04-03,6590.0,6790.0,6798.0,6541.0,6474.0,6562.0,1358096.0,602661.0,1.357177e+11,6927.0,6020.0,6790.0,6791.0,6793.0,6794.0,6795.0,6789.0,6788.0,6787.0,6786.0,6785.0,22.0,103.0,40.0,8.0,50.0,30.0,41.0,15.0,76.0,34.0,0.048811
65777,AG2406,2024-04-03 14:59:59.500,2024-04-03,6590.0,6791.0,6798.0,6541.0,6474.0,6562.0,1358246.0,602594.0,1.357330e+11,6927.0,6020.0,6791.0,6792.0,6793.0,6794.0,6795.0,6788.0,6787.0,6786.0,6785.0,6784.0,90.0,1.0,40.0,12.0,50.0,41.0,15.0,76.0,34.0,24.0,0.048965


In [31]:
#由于日盘无法获取前一期夜盘数据，夜盘也无法获取前一期日盘的数据
def process_day(group, prev_period, back_period):
    """
    按照日盘和夜盘时间范围划分交易数据，并对每段数据进行前后切片处理，最后返回拼接结果。

    :param group: 包含交易数据的 DataFrame，必须有 `trading_date` 和 `exchange_time` 列。
    :param prev_period: 去掉每段数据前 prev_period 条记录。
    :param back_period: 去掉每段数据后 back_period 条记录。
    :return: 处理后的日盘和夜盘数据拼接后的 DataFrame。
    """
    # 获取交易日期
    trading_date = group['trading_date'].iloc[0].strftime('%Y-%m-%d')

    # 定义日盘的时间范围
    day_start = pd.to_datetime(f"{trading_date} 09:00:00")
    day_end = pd.to_datetime(f"{trading_date} 15:00:00")

    # 定义夜盘的时间范围（包含跨天逻辑）
    night_start = pd.to_datetime(f"{trading_date} 21:00:00") - pd.Timedelta(days=1)
    night_end = pd.to_datetime(f"{trading_date} 02:30:00") 

    # 筛选日盘数据
    day_session = group[(group['exchange_time'] >= day_start) & (group['exchange_time'] <= day_end)]

    # 筛选夜盘数据
    night_session = group[(group['exchange_time'] >= night_start) & (group['exchange_time'] <= night_end)]

    # 对每段数据应用 prev_period 和 back_period 切片
    day_session_processed = day_session.iloc[prev_period:-back_period]
    night_session_processed = night_session.iloc[prev_period:-back_period]
    # 拼接处理后的日盘和夜盘数据
    processed_data = pd.concat([night_session_processed, day_session_processed], ignore_index=True)

    return processed_data


In [32]:
new_table = table.groupby('trading_date').apply(process_day, prev_period=120, back_period=120)

In [33]:
new_table

order_book_id           exchange_time trading_date    open  \
trading_date                                                                    
2024-04-08   0            AG2406 2024-04-08 09:01:00.500   2024-04-08  6879.0   
             1            AG2406 2024-04-08 09:01:01.000   2024-04-08  6879.0   
             2            AG2406 2024-04-08 09:01:01.500   2024-04-08  6879.0   
             3            AG2406 2024-04-08 09:01:02.000   2024-04-08  6879.0   
             4            AG2406 2024-04-08 09:01:02.500   2024-04-08  6879.0   
...                          ...                     ...          ...     ...   
             25391        AG2406 2024-04-08 14:58:58.000   2024-04-08  6879.0   
             25392        AG2406 2024-04-08 14:58:58.500   2024-04-08  6879.0   
             25393        AG2406 2024-04-08 14:58:59.000   2024-04-08  6879.0   
             25394        AG2406 2024-04-08 14:58:59.500   2024-04-08  6879.0   
             25395        AG2406 2024-04-08 14:59:00.000   2024-04-08  6879.0   

                      last    high     low  prev_settlement  prev_close  \
trading_date                                                              
2024-04-08   0      6817.0  6888.0  6802.0           6662.0      6791.0   
             1      6817.0  6888.0  6802.0           6662.0      6791.0   
             2      6817.0  6888.0  6802.0           6662.0      6791.0   
             3      6818.0  6888.0  6802.0           6662.0      6791.0   
             4      6818.0  6888.0  6802.0           6662.0      6791.0   
...                    ...     ...     ...              ...         ...   
             25391  7127.0  7128.0  6802.0           6662.0      6791.0   
             25392  7128.0  7128.0  6802.0           6662.0      6791.0   
             25393  7128.0  7128.0  6802.0           6662.0      6791.0   
             25394  7127.0  7128.0  6802.0           6662.0      6791.0   
             25395  7127.0  7128.0  6802.0           6662.0      6791.0   

                       Volume  OpenInterest      Turnover  limit_up  \
trading_date                                                          
2024-04-08   0        39664.0      599533.0  4.070813e+09    7128.0   
             1        39813.0      599588.0  4.086048e+09    7128.0   
             2        39980.0      599640.0  4.103123e+09    7128.0   
             3        40177.0      599615.0  4.123267e+09    7128.0   
             4        40337.0      599529.0  4.139630e+09    7128.0   
...                       ...           ...           ...       ...   
             25391  1043032.0      606079.0  1.098879e+11    7128.0   
             25392  1043052.0      606067.0  1.098901e+11    7128.0   
             25393  1043054.0      606067.0  1.098903e+11    7128.0   
             25394  1043063.0      606071.0  1.098912e+11    7128.0   
             25395  1043064.0      606072.0  1.098914e+11    7128.0   

                    limit_down  AskPrice1  AskPrice2  AskPrice3  AskPrice4  \
trading_date                                                                 
2024-04-08   0          6195.0     6817.0     6818.0     6819.0     6820.0   
             1          6195.0     6818.0     6819.0     6820.0     6821.0   
             2          6195.0     6817.0     6818.0     6819.0     6820.0   
             3          6195.0     6818.0     6819.0     6820.0     6821.0   
             4          6195.0     6818.0     6819.0     6820.0     6821.0   
...                        ...        ...        ...        ...        ...   
             25391      6195.0     7128.0        0.0        0.0        0.0   
             25392      6195.0     7128.0        0.0        0.0        0.0   
             25393      6195.0     7128.0        0.0        0.0        0.0   
             25394      6195.0     7128.0        0.0        0.0        0.0   
             25395      6195.0     7128.0        0.0        0.0        0.0   

                    AskPrice5  BidPrice1  BidPrice